In [1]:
import io
import random
import string
import warnings
import numpy as np
import os
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import cosine_similarity
warnings.filterwarnings('ignore')

In [2]:
import nltk
from nltk.stem import WordNetLemmatizer

In [3]:
with open('jaipur.txt', 'r', errors='ignore') as file:
    raw = file.read().lower()

nltk.download('wordnet', quiet=True)

sent_tokens = nltk.sent_tokenize(raw)

word_tokens = nltk.word_tokenize(raw)

In [4]:
sent_tokens[:2]

['jaipur is in the state of rajasthan, towards the north-western part of india.',
 'rajasthan shares proximity withgujarat, madhya pradesh, uttar pradesh, punjab, and haryana.']

In [5]:
lemmer = WordNetLemmatizer()

def LemTokens(tokens):
    return [lemmer.lemmatize(token) for token in tokens]

remove_punct_dict = dict((ord(punct), None) for punct in string.punctuation)

def LemNormalize(text):
    return LemTokens(nltk.word_tokenize(text.lower().translate(remove_punct_dict)))

In [6]:
# result = LemNormalize("Jaipur is running best")
# result

In [7]:
GREETING_INPUTS = ['hello', 'hi', 'greetings', 'sup', 'hey', 'hii']
GREETING_RESPONSES = ['hi', 'hello', 'hey', 'hi there', 'I am glad! You are talking to me']

def greeting(sentence):
    for word in sentence.split():
        if word.lower() in GREETING_INPUTS:
            return random.choice(GREETING_RESPONSES)

In [8]:
# stri = ""
# for i in LemNormalize("HI guys"):
#     stri+= i+" "
    
# result = greeting(stri)
# result

In [9]:
def response(user_response):
    robo_response = ''
    sent_tokens.append(user_response)
    
    
    TfidfVec = TfidfVectorizer(tokenizer=LemNormalize, stop_words='english')
    tfidf = TfidfVec.fit_transform(sent_tokens)
    
    vals = cosine_similarity(tfidf[-1], tfidf)
    idx = vals.argsort()[0][-2]
    
    flat = vals.flatten()
    flat.sort()
#     print(flat)
    
    req_tfidf = flat[-2]
    
    if(req_tfidf==0):
        robo_response += "I cannot understand you..."
        return robo_response
    else:
        robo_response += sent_tokens[idx]
        return robo_response

In [10]:
flag = True

print("Robo : I am here...")

while(flag):
    user_response = input().lower()
    if user_response!='bye':
        if user_response=='thanks' or user_response=='thank you':
            flag = False
            print("I am happy to answer your questions")
        else:
            greet = greeting(user_response)
            if greet!=None:
                print("Robo : ", greet)
            else:
                reply = response(user_response)
                print(reply)
                sent_tokens.remove(user_response)
                
    else:
        flag=False
        print("Robo : Bye!!")
    print()

Robo : I am here...
bye
Robo : Bye!!



In [11]:
from flask import Flask, render_template, request
app = Flask(__name__)

In [12]:
@app.route("/")
def basic():
    return render_template("chatbot.html")

@app.route("/get")
def get_bot_response():
    user_response = request.args.get("msg")
    user_response = user_response.lower()
    if user_response!='bye':
        if user_response=='thanks' or user_response=='thank you':
            flag = False
            print("I am happy to answer your questions")
            return str("I am happy to answer your questions")
        else:
            greet = greeting(user_response)
            if greet!=None:
                print("Robo : ", greet)
                return greet
            else:
                reply = response(user_response)
                print(reply)
                sent_tokens.remove(user_response)
                return reply
                
    else:
        flag=False
        print("Robo : Bye!!")
    

In [ ]:
if __name__=="__main__":
    app.run()

 * Serving Flask app "__main__" (lazy loading)
 * Environment: production
   Use a production WSGI server instead.
 * Debug mode: off


 * Running on http://127.0.0.1:5000/ (Press CTRL+C to quit)
127.0.0.1 - - [01/Dec/2020 20:00:09] "GET / HTTP/1.1" 200 -
127.0.0.1 - - [01/Dec/2020 20:00:11] "GET /favicon.ico HTTP/1.1" 404 -
127.0.0.1 - - [01/Dec/2020 20:01:29] "GET /get?msg=hey HTTP/1.1" 200 -


Robo :  hey


127.0.0.1 - - [01/Dec/2020 20:01:33] "GET /get?msg=hi HTTP/1.1" 200 -


Robo :  hello


...

jaipur is the capital of rajasthan.


127.0.0.1 - - [01/Dec/2020 20:10:18] "GET /get?msg=how%20is%20rajas HTTP/1.1" 200 -


I cannot understand you...


127.0.0.1 - - [01/Dec/2020 20:10:32] "GET /get?msg=where%20is%20rajasthan HTTP/1.1" 200 -


jaipur is the capital of rajasthan.
